In [1]:
import music21
little_star="tinynotation: 4/4 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2 g4 g4 f4 f4 e4 e4 d2 g4 g4 f4 f4 e4 e4 d2 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2"
music21.converter.parse(little_star).show('mid')

In [2]:
# 데이터  sampling
# ABC 표기 데이터는 2채널(계이름과 박자), 분류문제
# x : [ [1,4],[1,4],[1,4] ... ]
# y : [4 4]- 원핫 000000000100000

In [3]:
import numpy as np

In [8]:
temp = ['c','d','e','f','g','a','b']
note2num = dict(zip(temp,np.arange(1,8)))
num2note =  { value:key for key,value in note2num.items()  }
note2num, num2note

({'c': 1, 'd': 2, 'e': 3, 'f': 4, 'g': 5, 'a': 6, 'b': 7},
 {1: 'c', 2: 'd', 3: 'e', 4: 'f', 5: 'g', 6: 'a', 7: 'b'})

In [13]:
# 시계열 데이터 만들기
def abc2timeseries(s):
    return [ [note2num[i[0]], int(i[1]) ] for i in s.split()[2:] ]
temp = abc2timeseries(little_star)
temp[:5]

[[1, 4], [1, 4], [5, 4], [5, 4], [6, 4]]

In [17]:
def timeseries2abc(t):
    return "tinynotation: 4/4 "+" ".join([num2note[i]+str(j) for i,j in t])
timeseries2abc(temp)    

'tinynotation: 4/4 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2 g4 g4 f4 f4 e4 e4 d2 g4 g4 f4 f4 e4 e4 d2 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2'

In [19]:
# 원핫 코드로 변환하는 표
#[  [1,2],[2,2],..[7,2], ]
onehot = [[i,2] for i in range(1,8)] + [[i,4] for i in range(1,8)] + [[i,8] for i in range(1,8)]

[[1, 2],
 [2, 2],
 [3, 2],
 [4, 2],
 [5, 2],
 [6, 2],
 [7, 2],
 [1, 4],
 [2, 4],
 [3, 4],
 [4, 4],
 [5, 4],
 [6, 4],
 [7, 4],
 [1, 8],
 [2, 8],
 [3, 8],
 [4, 8],
 [5, 8],
 [6, 8],
 [7, 8]]

In [21]:
seq = abc2timeseries(little_star)
seq[:5]

[[1, 4], [1, 4], [5, 4], [5, 4], [6, 4]]

In [25]:
# 시계열 데이터 만들기
w = 8  # 윈도우 크기
h = 1  # 수평 계수
def seq2dadtaset(seq,w,h):
    
    for i in range(len(seq)-(w+h)+1):        
        x = seq[i:(i+w)]
        y = seq[i+w+h-1]
    

[[1, 4], [1, 4], [5, 4], [5, 4], [6, 4], [6, 4], [5, 2], [4, 4]] [4, 4]
[[1, 4], [5, 4], [5, 4], [6, 4], [6, 4], [5, 2], [4, 4], [4, 4]] [3, 4]
[[5, 4], [5, 4], [6, 4], [6, 4], [5, 2], [4, 4], [4, 4], [3, 4]] [3, 4]
[[5, 4], [6, 4], [6, 4], [5, 2], [4, 4], [4, 4], [3, 4], [3, 4]] [2, 4]
[[6, 4], [6, 4], [5, 2], [4, 4], [4, 4], [3, 4], [3, 4], [2, 4]] [2, 4]
[[6, 4], [5, 2], [4, 4], [4, 4], [3, 4], [3, 4], [2, 4], [2, 4]] [1, 2]
[[5, 2], [4, 4], [4, 4], [3, 4], [3, 4], [2, 4], [2, 4], [1, 2]] [5, 4]
[[4, 4], [4, 4], [3, 4], [3, 4], [2, 4], [2, 4], [1, 2], [5, 4]] [5, 4]
[[4, 4], [3, 4], [3, 4], [2, 4], [2, 4], [1, 2], [5, 4], [5, 4]] [4, 4]
[[3, 4], [3, 4], [2, 4], [2, 4], [1, 2], [5, 4], [5, 4], [4, 4]] [4, 4]
[[3, 4], [2, 4], [2, 4], [1, 2], [5, 4], [5, 4], [4, 4], [4, 4]] [3, 4]
[[2, 4], [2, 4], [1, 2], [5, 4], [5, 4], [4, 4], [4, 4], [3, 4]] [3, 4]
[[2, 4], [1, 2], [5, 4], [5, 4], [4, 4], [4, 4], [3, 4], [3, 4]] [2, 2]
[[1, 2], [5, 4], [5, 4], [4, 4], [4, 4], [3, 4], [3, 4], [2, 2]]